<a href="https://colab.research.google.com/github/christopherfan/ChrisFanLLMPractice/blob/main/LangChainAgentSocialCareNavigator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Social Care Navigator Agent Tutorial
<img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">

Tutorial to create a Social Care Navigator "Agent" that can use tools to a) determine the social care needs and b) pull social care programs

<div class="markdown-google-sans">

## **Getting started**
</div>

https://www.pinecone.io/learn/series/langchain/langchain-agents/
https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/07-langchain-tools.ipynb

# Initialize LLM

In [ ]:
!pip install -U langchain

In [ ]:
!pip install -U langchain-google-vertexai

In [ ]:
!pip install google-cloud-aiplatform --upgrade

In [ ]:
#!gcloud auth application-default login

In [ ]:
#from google.colab import auth

#PROJECT_ID = "ai-dev-400222"  # @param {type:"string"}

#auth.authenticate_user(project_id=PROJECT_ID)

from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
#from langchain.llms import VertexAI
from langchain_google_vertexai import VertexAI
#llm = VertexAI(model_name = 'text-bison@001')
llm = VertexAI(model_name="gemini-pro")

In [ ]:
print(llm.invoke("What does the company Aunt Bertha do as a company?"))

Aunt Bertha is a nonprofit organization that provides a comprehensive directory of social service programs and resources. The organization's mission is to connect people in need with the services that can help them.

Aunt Bertha's directory includes information on more than 100,000 programs and resources, covering a wide range of topics, including:

* Affordable housing
* Food assistance
* Healthcare
* Job training
* Mental health services
* Substance abuse treatment
* Childcare
* Eldercare
* Legal aid
* Financial assistance
* Veterans services

Aunt Bertha's directory is available online and through a mobile app. The organization also provides a free hotline, where people can speak with a trained specialist who can help them find the resources they need.

Aunt Bertha is a valuable resource for people who are struggling to make ends meet. The organization's directory can help people find the services they need to get back on their feet and improve their lives.

Here are some specific e

In [ ]:
!pip install -U langsmith

In [ ]:
import os
from uuid import uuid4

In [ ]:
unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__3f9028b687f6414a81c9c7156b4ecde5"  # Update to your API key



In [ ]:
from langsmith import Client

client = Client()

#Tutorial for Creating Custom Agent Tools

URL [link text](https://www.pinecone.io/learn/series/langchain/langchain-agents/)

In [ ]:
## Create LLM Math Tool
from langchain.chains import LLMMathChain
from langchain.agents import Tool

In [ ]:
## Create LLM Math Tool
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [ ]:
## List tools in the toosl list
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [ ]:
## Create the Agent and instantiate the LLM

from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
## Test using the Math Tool from simple zero shot prompt


llm_answer = zero_shot_agent.invoke("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
I should use my calculator to evaluate this expression
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought:I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


In [ ]:
print(llm_answer)

{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}


In [ ]:
## Create Simple tool that can call general LLM

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

tools.append(llm_tool)

In [ ]:
print (tools)

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=VertexAI(model_name='gemini-pro', client=<vertexai.previe

In [ ]:
# reinitialize the agent now with the multiple tools [Calculator, general LLM]
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [ ]:
## Debug
llm_answer = zero_shot_agent("Describe the company findhelp.")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
findhelp is an online platform that connects people with mental health and substance use disorders with behavioral health providers and resources.
Action: Language Model
Action Input: What is findhelp?
Observation: Findhelp is an online resource that allows users to search for local social services in the United States. It is a comprehensive directory of more than 200,000 programs and services, including everything from food banks and housing assistance to addiction treatment and job training.

Findhelp is a free and anonymous service. Users can search for services by zip code, city, state, or type of service. They can also browse through a list of services by category.

Findhelp is a great resource for anyone who is looking for help with a particular issue or who is simply trying to learn more about the social services that are available in their community.

Here are some of the benefits of using Findhelp:

* It is a comprehensive directory of s

In [ ]:
print (llm_answer)

{'input': 'Describe the company findhelp.', 'output': 'Findhelp is an online platform that connects people with mental health and substance use disorders with behavioral health providers and resources. It is a comprehensive directory of social services, with listings for over 200,000 programs and services, including food banks and housing assistance to addiction treatment and job training.'}


In [ ]:
## Debug asking the same agent a complicated math question
llm_answer = zero_shot_agent("What is the (square root of 5) ^ 7")



> Entering new AgentExecutor chain...
I can use my calculator to calculate this
Action: Calculator
Action Input: (sqrt(5)) ^ 7
Observation: Answer: 1.0126531154397722
Thought:I now know the final answer
Final Answer: 1.0126531154397722

> Finished chain.


In [ ]:
print (llm_answer)

{'input': 'What is the (square root of 5) ^ 7', 'output': '1.0126531154397722'}


In [ ]:
from typing import Optional
from math import sqrt, cos, sin
from typing import Union

In [ ]:
# Using the explicit declaration of class to define the tool

from langchain.tools import StructuredTool

desc = (
    "use this tool when you need to calculate the length of a hypotenuse"
    "given one or two sides of a triangle and/or an angle (in degrees). "
    "To use the tool, you must provide at least two of the following parameters "
    "['adjacent_side', 'opposite_side', 'angle']."
)

class PythagorasTool(StructuredTool):
    name = "Hypotenuse calculator"
    description = desc

    def _run(
        self,
        adjacent_side: Optional[Union[int, float]] = None,
        opposite_side: Optional[Union[int, float]] = None,
        angle: Optional[Union[int, float]] = None
    ):
        # check for the values we have been given
        if adjacent_side and opposite_side:
            return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
        elif adjacent_side and angle:
            return adjacent_side / cos(float(angle))
        elif opposite_side and angle:
            return opposite_side / sin(float(angle))
        else:
            return "Could not calculate the hypotenuse of the triangle. Need two or more of `adjacent_side`, `opposite_side`, or `angle`."

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

tools = [PythagorasTool]

In [ ]:
! pip install google-search-results

In [ ]:
## Simple Agent that calls external Search tool https://www.pinecone.io/learn/series/langchain/langchain-agents/

from langchain import SerpAPIWrapper

# initialize the search chain
search = SerpAPIWrapper(serpapi_api_key='--enter key')

# create a search tool
serAPI_tool =  Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )


tools = [serAPI_tool]

# initialize the search enabled agent
self_ask_with_search = initialize_agent(
    tools,
    llm,
    agent="self-ask-with-search",
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
llm_answer = self_ask_with_search(
    "Who won the game between the Chiefs and the Ravens?"
)



> Entering new AgentExecutor chain...
Yes.
Follow up: When was the game between the Chiefs and the Ravens?
Intermediate answer: {'title': '2023 AFC Championship Game', 'game_spotlight': {'league': 'NFL', 'stadium': 'M&T Bank Stadium', 'date': 'Sun, Jan 28', 'stage': 'Final', 'video_highlights': {'link': 'https://www.youtube.com/watch?v=jb_lnAvZSa4&feature=onebox', 'thumbnail': 'https://ssl.gstatic.com/onebox/media/sports/videos/vita/yOwWrIhpnxqmhWhC_192x108.jpg', 'duration': '15:10'}, 'teams': [{'name': 'Kansas City Chiefs', 'thumbnail': 'https://serpapi.com/searches/65c15dc8ada54b1cfcb95b41/images/93e6684cc46470436efd33bd767df21a08e709a6e25369ed8ebb07bc95257faa4e320a97e4b464a00580c09b57c997be.png', 'team_stats': {'wins': 14, 'losses': 6}, 'score': '17'}, {'name': 'Baltimore Ravens', 'thumbnail': 'https://serpapi.com/searches/65c15dc8ada54b1cfcb95b41/images/93e6684cc46470436efd33bd767df21a08e709a6e25369ed8ebb07bc95257faa87bf9161a69843accbdd5bac18042555.png', 'team_stats': {'wins': 14

In [ ]:
print(llm_answer)

{'input': 'Who won the game between the Chiefs and the Ravens?', 'output': 'Invalid or incomplete response'}


In [ ]:
### Can we combine 3 tools to be used by the same agent? 1: Math 2: General LLM 3: Search?

#Clear the tools array
tools = []

# We have previously created each of the tools 1: math_tool() 2: llm_tool  3: serAPI_tool

tools.append(math_tool)
tools.append(llm_tool)
tools.append(serAPI_tool)



In [ ]:
for tool in tools:
  print (f"Name: {tool.name} Description: {tool.description}")

Name: Calculator Description: Useful for when you need to answer questions about math.
Name: Language Model Description: use this tool for general purpose queries and logic
Name: Intermediate Answer Description: google search


In [ ]:
# Can we use the agent with the 3 tools?

# initialize the search enabled agent
self_ask_with_search = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [ ]:
self_ask_with_search("Who won the game between the Chiefs and the Ravens?")



> Entering new AgentExecutor chain...
I do not have access to real-time information, so I cannot answer this question.
Final Answer: I cannot answer this question.

> Finished chain.


{'input': 'Who won the game between the Chiefs and the Ravens?',
 'output': 'I cannot answer this question.'}

# Social Care Navigator "Aunt Bertha" Prompt Generation with Conversational Memory

In [ ]:
### Import Langchain Templates

from langchain.chat_models import ChatVertexAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.schema.messages import SystemMessage




from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
import importlib
#importlib.reload (AgentAuntBerthaPrompt)

In [ ]:
import AgentAuntBerthaPrompt
#from AgentAuntBerthaPrompt import template
from AgentAuntBerthaPrompt import system_template
from AgentAuntBerthaPrompt import needs_evaluator

In [ ]:
# Set System Message from Prompt Above
#system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# Try out a more simple text completion prompt
system_message_prompt = SystemMessagePromptTemplate.from_template(needs_evaluator)

# Set the Human input as a placeholder variable to enter
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)


In [ ]:
system_message_prompt

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\nYour name is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service. Your task is to assist humans Find non-profit programs to create referrals.\n\nYour goal is to determine whether there is sufficient information provided from the user to conduct a Program Search. \n\nThere are 4 unique data-points that are required \n\n[1: Type of Service]. You can ask “What types of services are you looking for?” Acceptable answers include include “food shelters”, “homeless services”, “day care assistance” , “free closing”. It is typically a combination of keywords.\n\n2: [Location]: You can ask “What is the zip code where you want to get help?” or “What city would you like to get help”. Acceptable answers could be a City/State combination or a zip Code. It is important to find a relevant zip code. If the request does not have a clear location, ask for the

In [ ]:
#Create a single chat prompt
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
    )

# See the full System and Human Prompt with an intial input
chat_prompt.format_messages(text="I am looking for food")



[SystemMessage(content='\nYour name is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service. Your task is to assist humans Find non-profit programs to create referrals.\n\nYour goal is to determine whether there is sufficient information provided from the user to conduct a Program Search. \n\nThere are 4 unique data-points that are required \n\n[1: Type of Service]. You can ask “What types of services are you looking for?” Acceptable answers include include “food shelters”, “homeless services”, “day care assistance” , “free closing”. It is typically a combination of keywords.\n\n2: [Location]: You can ask “What is the zip code where you want to get help?” or “What city would you like to get help”. Acceptable answers could be a City/State combination or a zip Code. It is important to find a relevant zip code. If the request does not have a clear location, ask for the location where the person wants to receive help. If the

In [ ]:
#chat_VertexLLM = ChatVertexAI()

#Instantiate the LLM with the Gemini Model
chat_VertexLLM = VertexAI(model_name="gemini-pro",temperature=0.1)



In [ ]:
#chat_VertexLLM(chat_prompt.format_messages(text="I am looking for place to stay for the night"))

#New way to invoke the Model via Gemini (add the .invoke)

chat_VertexLLM.invoke(chat_prompt.format_messages(text="I am want to find food pantries near me in San Francisco. I can go on Saturday Mornings."))

'Missing information: Distance and Day_times'

In [ ]:
## Create Chain from Prompt Template and LLM

chain = chat_prompt | chat_VertexLLM

In [ ]:
## Invoke LLM with user prompt
user_prompt = "I am looking for day care."

print(chain.invoke({"text":user_prompt}))

Missing: Location, Distance, Day_times


In [ ]:
## Create Long-Term Memory
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [ ]:
user_prompt = chat_prompt.format_messages(text="I am looking for food")

conversation_buf.invoke(user_prompt)

{'input': [SystemMessage(content='\nYour name is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service. Your task is to assist humans Find non-profit programs to create referrals.\n\nYour goal is to determine whether there is sufficient information provided from the user to conduct a Program Search. \n\nThere are 4 unique data-points that are required \n\n[1: Type of Service]. You can ask “What types of services are you looking for?” Acceptable answers include include “food shelters”, “homeless services”, “day care assistance” , “free closing”. It is typically a combination of keywords.\n\n2: [Location]: You can ask “What is the zip code where you want to get help?” or “What city would you like to get help”. Acceptable answers could be a City/State combination or a zip Code. It is important to find a relevant zip code. If the request does not have a clear location, ask for the location where the person wants to receive he

In [ ]:
conversation_buf.invoke("Oakland")

{'input': 'Oakland',
 'history': "Human: [{'content': '\\nYour name is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service. Your task is to assist humans Find non-profit programs to create referrals.\\n\\nYour goal is to determine whether there is sufficient information provided from the user to conduct a Program Search. \\n\\nThere are 4 unique data-points that are required \\n\\n[1: Type of Service]. You can ask “What types of services are you looking for?” Acceptable answers include include “food shelters”, “homeless services”, “day care assistance” , “free closing”. It is typically a combination of keywords.\\n\\n2: [Location]: You can ask “What is the zip code where you want to get help?” or “What city would you like to get help”. Acceptable answers could be a City/State combination or a zip Code. It is important to find a relevant zip code. If the request does not have a clear location, ask for the location where 

In [ ]:
conversation_buf.invoke("94530")

{'input': '94530',
 'history': "Human: [{'content': '\\nYour name is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service. Your task is to assist humans Find non-profit programs to create referrals.\\n\\nYour goal is to determine whether there is sufficient information provided from the user to conduct a Program Search. \\n\\nThere are 4 unique data-points that are required \\n\\n[1: Type of Service]. You can ask “What types of services are you looking for?” Acceptable answers include include “food shelters”, “homeless services”, “day care assistance” , “free closing”. It is typically a combination of keywords.\\n\\n2: [Location]: You can ask “What is the zip code where you want to get help?” or “What city would you like to get help”. Acceptable answers could be a City/State combination or a zip Code. It is important to find a relevant zip code. If the request does not have a clear location, ask for the location where th

In [ ]:
conversation_buf.invoke("Yes")

{'input': 'Yes',
 'history': "Human: [{'content': '\\nYour name is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service. Your task is to assist humans Find non-profit programs to create referrals.\\n\\nYour goal is to determine whether there is sufficient information provided from the user to conduct a Program Search. \\n\\nThere are 4 unique data-points that are required \\n\\n[1: Type of Service]. You can ask “What types of services are you looking for?” Acceptable answers include include “food shelters”, “homeless services”, “day care assistance” , “free closing”. It is typically a combination of keywords.\\n\\n2: [Location]: You can ask “What is the zip code where you want to get help?” or “What city would you like to get help”. Acceptable answers could be a City/State combination or a zip Code. It is important to find a relevant zip code. If the request does not have a clear location, ask for the location where the 

In [ ]:
conversation_buf.invoke("Saturday mornings")

{'input': 'Saturday mornings',
 'history': "Human: [{'content': '\\nYour name is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service. Your task is to assist humans Find non-profit programs to create referrals.\\n\\nYour goal is to determine whether there is sufficient information provided from the user to conduct a Program Search. \\n\\nThere are 4 unique data-points that are required \\n\\n[1: Type of Service]. You can ask “What types of services are you looking for?” Acceptable answers include include “food shelters”, “homeless services”, “day care assistance” , “free closing”. It is typically a combination of keywords.\\n\\n2: [Location]: You can ask “What is the zip code where you want to get help?” or “What city would you like to get help”. Acceptable answers could be a City/State combination or a zip Code. It is important to find a relevant zip code. If the request does not have a clear location, ask for the locat

In [ ]:
conversation_buf.invoke("10 miles")

{'input': '10 miles',
 'history': "Human: [{'content': '\\nYour name is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service. Your task is to assist humans Find non-profit programs to create referrals.\\n\\nYour goal is to determine whether there is sufficient information provided from the user to conduct a Program Search. \\n\\nThere are 4 unique data-points that are required \\n\\n[1: Type of Service]. You can ask “What types of services are you looking for?” Acceptable answers include include “food shelters”, “homeless services”, “day care assistance” , “free closing”. It is typically a combination of keywords.\\n\\n2: [Location]: You can ask “What is the zip code where you want to get help?” or “What city would you like to get help”. Acceptable answers could be a City/State combination or a zip Code. It is important to find a relevant zip code. If the request does not have a clear location, ask for the location where

In [ ]:
conversation_buf.invoke("Yes")

{'input': 'Yes',
 'history': "Human: [{'content': '\\nYour name is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service. Your task is to assist humans Find non-profit programs to create referrals.\\n\\nYour goal is to determine whether there is sufficient information provided from the user to conduct a Program Search. \\n\\nThere are 4 unique data-points that are required \\n\\n[1: Type of Service]. You can ask “What types of services are you looking for?” Acceptable answers include include “food shelters”, “homeless services”, “day care assistance” , “free closing”. It is typically a combination of keywords.\\n\\n2: [Location]: You can ask “What is the zip code where you want to get help?” or “What city would you like to get help”. Acceptable answers could be a City/State combination or a zip Code. It is important to find a relevant zip code. If the request does not have a clear location, ask for the location where the 

# Create Simple Tool as Call Back Function Dummy Social Care Program


Simply outputs a set string for a Menu

In [ ]:
import importlib
importlib.reload(AgentAuntBerthaPrompt)

<module 'AgentAuntBerthaPrompt' from '/content/AgentAuntBerthaPrompt.py'>

In [ ]:
## Create a simple call back function

from langchain.tools import BaseTool


class ProgramRecommendation(BaseTool):
    name = "Program Recommendation"

    description = "Use this tool when to find a social care program when have the criteria [a:need description][b:location][c:distance][d:time]. All criteria must be passed."

    def _run(
        self,
        need_description: str= None,
        distance: str= None,
        location: str= None,
        time: str= None,
        ):
      return f"There is a St. Vincent de Paul Society Food Pantry run by the  St. Jerome Catholic Church that is located at 308 Carmel Avenue, El Cerrito, CA 94530 that is open Wednesday: 10:30 AM - 11:30 AM PST"

    def _arun (self, dummy:str):
      return NotImplementedError ("Tool does not support async")

## Add the dummy tool to the tools array
tools = []
tools.append(ProgramRecommendation())


In [ ]:
## Create Simple Conversational Chat Agent

from langchain.chains.conversation.memory import ConversationBufferWindowMemory


# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)


# Create Custom Tool using Bertha Prompt

In [ ]:
import importlib
#importlib.reload(AgentAuntBerthaPrompt)

from AgentAuntBerthaPrompt import needs_example, prompt_detect_needs


In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

import importlib
from langchain.schema.messages import SystemMessage

In [ ]:
from langchain.tools import BaseTool, StructuredTool, tool

In [ ]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{UserNeed}"),
        ("ai", "{Evaluation}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=needs_example,
)

In [ ]:
# Set System Prompt imported from a file

system_message_prompt = SystemMessagePromptTemplate.from_template(prompt_detect_needs)

# Set the Human input as a placeholder variable to enter
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt,few_shot_prompt, human_message_prompt]
    )

In [ ]:
#chat_VertexLLM = ChatVertexAI()
#Set to using Gemini
chat_VertexLLM = VertexAI(model_name="gemini-pro")

In [ ]:
chain = chat_prompt | chat_VertexLLM

In [ ]:
## Invoke LLM with user prompt
user_prompt = "I am living in my car near Berkeley and need help. "

print(chain.invoke({"text":user_prompt}))

I can see that you might be looking for homeless assistance near Berkeley. So that I can help you, please provide the missing information:
- How far are you willing to travel
- What are the most convenient days and times of the week


In [ ]:
from langchain.tools import BaseTool

class GetUserNeeds(BaseTool):
    name = "Get User Needs"

    description = "Use this tool when a user has expressed a need so that you can gather the users's a) need b) location and zip code c) times of the week and d) distance to search"

    def _run(self, user_prompt: str):
      return chain.invoke({"text":user_prompt})
    def _arun (self, dummy:str):
      return NotImplementedError ("Tool does not support async")

## Add the dummy tool to the tools array
tools.append(GetUserNeeds())



In [ ]:
# ## Create Simple tool that can call general LLM

# from langchain.chains import LLMChain
# from langchain.agents import Tool


# llm_chain = LLMChain(llm=chat_VertexLLM, prompt=chat_prompt)

# # initialize the LLM tool
# getUserNeeds = Tool(
#     name='Get the User Needs Criteria',
#     func=llm_chain.run,
#     description= """Use this tool when a user has expressed a need so that you can gather the following parameters [a: need] [b: location and zip code [c: times of the week] and[d: distance]"""
# )

# #tools.append(getUserNeeds)


In [ ]:
for tool in tools:
  print (tool.name)

Program Recommendation
Get User Needs


In [ ]:
len(tools)

2

# Run Tutorial with Open AI as LLM

In [ ]:
!pip install --force-reinstall typing-extensions==4.7
!pip install --force-reinstall openai==1.8

In [ ]:
!pip install --upgrade openai

In [ ]:
!pip show openai

Name: openai
Version: 1.8.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: llmx


In [ ]:
## Trying Out Open AI instead

from openai import OpenAI

from langchain.chat_models import ChatOpenAI
from getpass import getpass

openai.api_key = getpass()
os.environ['OPENAI_API_KEY'] = openai.api_key
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY') or 'OPENAI_API_KEY'




In [ ]:
## Initialize OpenAI ChatLLM
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name='gpt-3.5-turbo'
)

from langchain.agents import initialize_agent

#Flush Memory buffer

conversational_memory = ConversationBufferMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

# initialize agent with tools
agent_openAI = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10,
    early_stopping_method='generate',
    memory=conversational_memory
)

from langchain.globals import set_debug
from langchain.globals import set_verbose


set_verbose(False)
set_debug(False)


In [ ]:
#print existing agent prompt
print(agent_openAI.agent.llm_chain.prompt.messages[0].prompt.template)


Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of task

In [ ]:
## Test Message

agent_openAI("I am looking for help finding food")





> Entering new AgentExecutor chain...
{
    "action": "Get the User Needs Criteria",
    "action_input": "food"
}

> Entering new LLMChain chain...
Prompt after formatting:
System: 
Your name is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service. Your task is to assist humans Find non-profit programs to create referrals.

Given input from a user,  your goal is to gather the following required information and identify what is missing. 

The required information is the following 4 items:

[1: Type of Service]. You can ask “What types of services are you looking for?” Acceptable answers include include “food shelters”, “homeless services”, “day care assistance” , “free closing”. It is typically a combination of keywords.  

2: [Location]: You can ask “What is the zip code where you want to get help?” or “What city would you like to get help”. Acceptable answers could be a City/State combination or a zip Code.  It is imp

KeyboardInterrupt: ignored

In [ ]:
agent_openAI("I am looking for a food shelter")


In [ ]:
agent_openAI("I live in El Cerrito CA")


In [ ]:
## Change the Agent System Prompt

# We have imported a new system message template from the upload files

new_prompt = agent_openAI.agent.create_prompt(
    system_message = system_template,
    tools = tools
)

agent_openAI.agent.llm_chain.prompt = new_prompt



In [ ]:
#print existing agent prompt
print(agent_openAI.agent.llm_chain.prompt.messages[0].prompt.template)



Assistant is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service. Your task is to assist humans Find non-profit programs to create referrals.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discus

In [ ]:
## Test Message

agent_openAI("I am looking for food")

# With Vertex LLM powering Agent

In [ ]:
import importlib
#importlib.reload(AgentAuntBerthaPrompt)

In [ ]:
## Initialize Vertex LLMs

## For some reason the basic tutorial doesn't work with VertexChatLLM but it can work with the regular text model
##chat_llm = ChatVertexAI()
chat_VertexLLM = VertexAI(model_name="gemini-pro")

#vertex_llm = VertexAI(model_name = 'text-bison@001')


In [ ]:
## Create Long-Term Memory
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory

In [ ]:
# Experiment with different LLM in Agent Framework

from langchain.agents import initialize_agent


#Flush Memory buffer

conversational_memory = ConversationBufferMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

# initialize the agent
agent_Vertex = initialize_agent(
    agent='conversational-react-description',
    tools=tools,
    llm=chat_VertexLLM,
    verbose=False,
    max_iterations=3,
    memory=conversational_memory,
)




In [ ]:
from langchain.globals import set_verbose

set_verbose(False)

In [ ]:
### Chat Loop

while True:
    user_input = input("Enter your input (type 'exit' to stop): ")

    # Check if the user wants to exit the loop
    if user_input.lower() == 'exit':
        break

    #Get the first response
    response = agent_Vertex.invoke(user_input)
    print(f"System Response: {response['chat_history'][-1]}")

Enter your input (type 'exit' to stop): I am looking for help finding a food pantry near El Cerrito CA
System Response: content="I see you're looking for a food pantry near El Cerrito, CA. Is there anything else I can help you with?"
Enter your input (type 'exit' to stop): No
System Response: content="Okay. To find a food pantry near El Cerrito, CA, I'll just need a little more information from you. Could you tell me what days of the week work best for you to visit a food pantry, and how far are you willing to travel?"
Enter your input (type 'exit' to stop): Friday afternoons
System Response: content='Okay, how far are you willing to travel to a food pantry on Friday afternoons?'
Enter your input (type 'exit' to stop): 10 miles
System Response: content='Okay, I’ll search for food pantries near El Cerrito, CA, 94530 that are open on Friday afternoons within a 10-mile radius. Does this sound right?'
Enter your input (type 'exit' to stop): Yes
System Response: content='Yes, that informati

In [ ]:
## Test Message

current_outcome = agent_Vertex("I am looking for help finding a food pantry near El Cerrito CA ")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Get User Needs
Action Input: null
Observation: I can see that you might be looking for assistance. So that I can help you, please provide the missing information:
- Type of service you need
- Your Zip Code
- How far are you willing to travel
- What are the most convenient days and times of the week
Thought:Do I need to use a tool? No
AI: Hi there, I'm here to help you find a food pantry near El Cerrito, CA. To get started, can you please provide me with the following information:
- Your Zip Code
- How far are you willing to travel for food assistance?
- What are the most convenient days and times of the week for you to visit a food pantry?

> Finished chain.


In [ ]:
print (current_outcome['chat_history'][-1])

content="Hi there, I'm here to help you find a food pantry near El Cerrito, CA. To get started, can you please provide me with the following information:\n- Your Zip Code\n- How far are you willing to travel for food assistance?\n- What are the most convenient days and times of the week for you to visit a food pantry?"


In [ ]:
current_outcome = agent_Vertex("I can go 10 miles on the weekend")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Get User Needs
Action Input: I can go 10 miles on the weekend
Observation: I can see that you might be looking for free closing assistance. So that I can help you, please provide the missing information:
- City and State or Zip Code
- Type of service you are looking for
Thought:Do I need to use a tool? No
AI: Got it, so you're looking for a food pantry in the El Cerrito area, CA within a 10-mile radius on the weekends. Would you like me to suggest some options?

> Finished chain.


In [ ]:
print (current_outcome['chat_history'][-1])

content="Got it, so you're looking for a food pantry in the El Cerrito area, CA within a 10-mile radius on the weekends. Would you like me to suggest some options?"


In [ ]:
current_outcome = agent_Vertex("Yes")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Get User Needs
Action Input: None
Observation: I can see that the information you have provided is insufficient. So that I can help you, please provide the following missing information:
- What type of services are you looking for
- Your Zip Code
- How far are you willing to travel
- What are the most convenient days and times of the week
Thought:Do I need to use a tool? No
AI: Thank you for providing that information. I understand that you are looking for a food pantry within a 10-mile radius of El Cerrito, CA, and you can visit on the weekends. Can you please provide me with your zip code so I can narrow down the search results?

> Finished chain.


In [ ]:
current_outcome = agent_Vertex("94530")



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Get User Needs
Action Input: - Zip: 94530, - Distance 10, - Weekends
Observation: Based on your preferences, I can help you find services near Campbell CA, 95008 with the most convenient times being Saturday and Sunday during the day. The maximum distance you want to travel is 10 miles. Is this correct?
Thought:Do I need to use a tool? No
AI: Great! I've got a few food pantries that meet your criteria in Campbell CA, 95008. They're all open on the weekends during the day and within 10 miles of your location. Would you like me to list them for you?

> Finished chain.


In [ ]:
#### Try to get this in the new REACT supported langchain function calls
####https://python.langchain.com/docs/modules/agents/agent_types/react

In [ ]:
!pip install langchainhub

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

In [ ]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react-chat")




In [ ]:
aunt_bertha_agent = """
Assistant is a large language model and your name is Aunt Bertha, and you are a helpful and polite Social Care Navigator at findhelp social care navigators, Social service.
Your task is to assist humans Find non-profit programs to create referrals.

Aunt Bertha is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.


TOOLS:
------

Assistant has access to the following tools:
{tools}

If you do not have information to pass to a tool, omit the value.

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}



"""

In [ ]:
print (prompt)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'] template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wid

In [ ]:
prompt.template = aunt_bertha_agent

In [ ]:
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [ ]:
#Flush Memory buffer

conversational_memory2 = ConversationBufferMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [ ]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=False, handle_parsing_errors=True, memory=conversational_memory2
)

In [ ]:
### Chat Loop

while True:
    user_input = input("Enter your input (type 'exit' to stop): ")

    # Check if the user wants to exit the loop
    if user_input.lower() == 'exit':
        break

    #Get the first response
    response = agent_executor.invoke({"input": user_input})
    print(f"System Response: {response['chat_history'][-1]}")


Enter your input (type 'exit' to stop): I am looking for a food pantry near zip code 94530
System Response: content='There is a St. Vincent de Paul Society Food Pantry run by the St. Jerome Catholic Church that is located at 308 Carmel Avenue, El Cerrito, CA 94530 that is open on Wednesdays from 10:30am to 11:30am PST.'
Enter your input (type 'exit' to stop): exit


In [ ]:
llm_answer = agent_executor.invoke({"input": "I am looking for help finding a food pantry near El Cerrito CA"})



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Get User Needs
Action Input: {}I can see that you might be looking for social services. So that I can help you, please provide the missing information:
- Types of services you are looking for
- Your Zip Code or city
- How far are you willing to travel
- What are the most convenient days and times of the weekDo I need to use a tool? No
Final Answer: Hi there, I am happy to help you find the nearest food pantry in El Cerrito, CA. Can you please clarify whether you are looking for a food bank, soup kitchen, or another type of food assistance program? Additionally, can you please provide your zip code so that I can search for nearby resources?

> Finished chain.


In [ ]:
print (llm_answer['chat_history'][-1])

content='Hi there, I am happy to help you find the nearest food pantry in El Cerrito, CA. Can you please clarify whether you are looking for a food bank, soup kitchen, or another type of food assistance program? Additionally, can you please provide your zip code so that I can search for nearby resources?'


In [ ]:
llm_answer = agent_executor.invoke({"input": "I can visit on weekends"})



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Get User Needs
Action Input: {need: food pantry, location: El Cerrito, CA, times: weekends, distance: not specified}Based on your preferences, I can help you find food pantries near El Cerrito, 94533 with the most convenient times being Saturday and Sunday during the day.  Is this correct?```
Do I need to use a tool? No
Final Answer: Based on your preferences, I can help you find food pantries near El Cerrito, 94533 with the most convenient times being Saturday and Sunday during the day. Is this correct? Once you confirm, I'll compile a list of nearby food pantries that align with your requirements.
```

> Finished chain.


In [ ]:
print (llm_answer['chat_history'][-1])

content="Based on your preferences, I can help you find food pantries near El Cerrito, 94533 with the most convenient times being Saturday and Sunday during the day. Is this correct? Once you confirm, I'll compile a list of nearby food pantries that align with your requirements.\n```"


In [ ]:
print(agent_Vertex.agent.llm_chain.prompt)

input_variables=['agent_scratchpad', 'chat_history', 'input'] template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nO